In [1]:
import os
import math
import trueskill as ts
from trueskill import Rating, quality_1vs1, rate_1vs1

In [2]:
BETA = ts.BETA
cdf = ts.TrueSkill().cdf

In [3]:
def draw_prob(muH,sigmaH,muA,sigmaA):
    try:
        hrate = Rating(muH,sigmaH)
        arate = Rating(muA,sigmaA)
        prob = quality_1vs1(hrate,arate)
        return prob
    except:
        return np.nan

In [4]:
def win_probability(muH,sigmaH,muA,sigmaA):
    try:
        a = [Rating(muH,sigmaH)]
        b = [Rating(muA,sigmaA)]  
        deltaMu = sum([x.mu for x in a]) - sum([x.mu for x in b])                  
        sumSigma = sum([x.sigma ** 2 for x in a]) + sum([x.sigma ** 2 for x in b])  
        playerCount = len(a) + len(b)                                               
        denominator = math.sqrt(playerCount * (BETA * BETA) + sumSigma)             
        return cdf(deltaMu / denominator)  
    except Exception as e:
        print(e)
        return np.nan

In [5]:
files = os.listdir('./trueskill_data_for_classifier/')
files.sort()

In [6]:
index = [int(f.split('_')[1].split('.')[0]) for f in files]

In [7]:
files_ar = pd.DataFrame(files, index=index).sort_index().values.reshape(-1)

In [15]:
dfs_combined = []
for p, file in enumerate(files_ar[:20]):
    print(file)
    df_trueskill = pd.read_csv('./trueskill_data_for_classifier/data_'+str(p)+'.csv', index_col='Unnamed: 0')
    df_elo = pd.read_csv('./elo_goals_data_for_classifier/data_'+str(p)+'.csv', index_col='Unnamed: 0')
    df_combined = pd.concat([df_trueskill,df_elo[['EH','EA']]], ignore_index=False, axis=1)
    dfs_combined.append(df_combined)

data_0.csv
data_1.csv
data_2.csv
data_3.csv
data_4.csv
data_5.csv
data_6.csv
data_7.csv
data_8.csv
data_9.csv
data_10.csv
data_11.csv
data_12.csv
data_13.csv
data_14.csv
data_15.csv
data_16.csv
data_17.csv
data_18.csv
data_19.csv


In [16]:
dfs_combined_final = pd.concat(dfs_combined,axis=0, ignore_index=True)

In [17]:
print('Draw')
dfs_combined_final['DrawProb'] = dfs_combined_final.apply(lambda row: draw_prob(row['muH'], row['sigmaH'],row['muA'], row['sigmaA']), axis=1)
print('Win')
dfs_combined_final['WinProb'] = dfs_combined_final.iloc[:].apply(lambda row: win_probability(row['muH'], row['sigmaH'],row['muA'], row['sigmaA']), axis=1)
print('Loss')
dfs_combined_final['LooseProb'] = 1 - dfs_combined_final['WinProb']

Draw
Win
Loss


In [18]:
dfs_combined_final.to_csv('./final_elo_and_trueskill.csv')

In [19]:
dfs_combined_final.shape

(941307, 12)

In [12]:
# http://sebastiandahlgren.se/2012/10/11/using-paramiko-to-send-ssh-commands/
# https://github.com/lucasmaystre/lsr/blob/master/simple-example.ipynb
# https://github.com/erdman/plackett-luce/blob/master/plackett_luce.py
# https://github.com/lucasmaystre/choix/blob/master/notebooks/intro-pairwise.ipynb
# https://stackoverflow.com/questions/16353729/pandas-how-to-use-apply-function-to-multiple-columns